### Step 1: Frame the problem

The first question to ask your stakeholder/manager/boss is what is the business objective. <br>
Questions such as: <br>
1. What will be the use case of the model? or What business problem are you trying to address?
2. How does the company expect to benefit from the model?
3. What metric will we use to call this project a success and how will we measure it?

#### Problem: Predict the destrict's median house price

Questions to ask: <br>
- Do we currently have the data required data or do we need to first start collecting it?
- If the data is availabl, how the data is collected and what are the data sources?
- If data is available, what sort of attributes/features would you need to actually start making the predictions?
- Is there any current solution in place or are we starting from the scratching?

#### What sort of problem is it?
It apears to be supervised learning problem and more specifically **multiple regression** problem.

#### Performance Measure
Root Mean Square Error

### Step 2: Download or Load the Dataset

In [2]:
from pathlib import Path
import pandas as pd
import tarfile
import urllib.request

def load_housing_data():
    tarball_path = Path("datasets/housing.tgz")
    if not tarball_path.is_file():
        Path("datasets").mkdir(parents=True, exist_ok=True)
        url = "https://github.com/ageron/data/raw/main/housing.tgz"
        urllib.request.urlretrieve(url, tarball_path)
        with tarfile.open(tarball_path) as housing_tarball:
            housing_tarball.extractall(path="datasets")
    return pd.read_csv(Path("datasets/housing/housing.csv"))

In [5]:
import pandas as pd

df = load_housing_data()

df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


### Step 2: Exploratory Data Analysis

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


### Step 3: Prepare Test dataset

In [9]:
import numpy as np

def shuffle_and_split_data(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

In [12]:
train_set, test_set = shuffle_and_split_data(df, 0.2)
print(f'The training dataset consists of {len(train_set)} rows.')
print(f'The test dataset consists of {len(test_set)} rows')

The training dataset consists of 16512 rows.
The test dataset consists of 4128 rows


In [39]:
train_set.dropna(inplace=True)

In [75]:
x_train = train_set.drop(columns=['median_house_value','ocean_proximity'])
y_train = train_set['median_house_value']

In [76]:
x_test = test_set.drop(columns=['median_house_value','ocean_proximity'])
x_test.dropna(inplace=True)

In [77]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2)

In [78]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error

linreg = LinearRegression()
linreg.fit(X_train, y_train)

LinearRegression()

In [79]:
print(linreg.intercept_)

-3526542.235225818


In [80]:
print(linreg.coef_)

[-4.19954543e+04 -4.17477581e+04  1.19693745e+03 -9.14217355e+00
  1.14029115e+02 -3.55344569e+01  4.66167419e+01  4.01470792e+04]


In [81]:
mse = mean_squared_error(y_test, linreg.predict(X_test))

np.sqrt(mse)

69372.78595028131

In [82]:
linreg.score(X_test,y_test)

0.6262240932118317

In [85]:
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [86]:
# using the model
linreg.predict(np.array([[-122.23, 37.88, 41, 880, 129 , 322, 126, 8.3252]]))

/Users/gagandeepkundi/opt/anaconda3/envs/ml/lib/python3.10/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([409560.22689403])